In [ ]:
# GENERAL IMPORTS

import pandas as pd
import numpy as np
import scipy as sp

import datetime
import math
import time

In [ ]:
infile = r'''Data\IVE_tickbidask.txt'''

colnames = ['Date', 'Time', 'TradedPrice', 'Bid', 'Ask', 'Volume']
tickdata = pd.read_csv(infile, names=colnames)

# tickdata['DateTime'] = (tickdata['Date']+tickdata['Time']).map(lambda x: datetime.datetime(int(x[6:10]), int(x[0:2]), int(x[3:5]), int(x[10:12]), int(x[13:15])))
# del tickdata['Date']
# del tickdata['Time']

tickdata['Date'] = tickdata['Date'].map(lambda x: datetime.date(int(x[6:10]), int(x[0:2]), int(x[3:5])))
tickdata['Time'] = tickdata['Time'].map(lambda x: datetime.time(int(x[0:2]), int(x[3:5])))

tickdata = tickdata[[(t >= datetime.time(9,30)) and (t < datetime.time(16,0)) for t in tickdata['Time']]].copy()

In [ ]:
minutedata = []

for day in tickdata.groupby('Date'):
    print(day[1]['Date'].iloc[0])
    for data in day[1].groupby('Time'):
        minutedata.append(
            {
                "Date": data[1]['Date'].iloc[0],
                "Time": data[1]['Time'].iloc[0],
                "BidOpen": data[1]['Bid'].iloc[0],
                "BidHigh": data[1]['Bid'].max(),
                "BidLow": data[1]['Bid'].min(),
                "BidClose": data[1]['Bid'].iloc[-1],
                "AskOpen": data[1]['Ask'].iloc[0],
                "AskHigh": data[1]['Ask'].max(),
                "AskLow": data[1]['Ask'].min(),
                "AskClose": data[1]['Ask'].iloc[-1],
                "Volume": data[1]['Volume'].sum()
            }
        )

In [ ]:
df = pd.DataFrame(minutedata)[['Date', 'Time', 'BidOpen', 'BidHigh', 'BidLow', 'BidClose', 'AskOpen', 'AskHigh', 'AskLow', 'AskClose', 'Volume']]
df['Date'] = df['Date'].map(lambda x: x.strftime("%m/%d/%Y"))
df['Time'] = df['Time'].map(lambda x: x.strftime("%H:%M"))

outfile = r'''Data\IVE_bidaskvol1min.txt'''
df.to_csv(outfile, index=False, header=False)